In [68]:
import cv2
import numpy as np
from os.path import isfile, join
from os import listdir
from random import shuffle
from shutil import copyfile
import os
import tensorflow as tf 
import datetime



class DataSetGenerator:
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.data_labels = self.get_data_labels()
        self.data_info = self.get_data_paths()

    def get_data_labels(self):
        data_labels = []
        for filename in listdir(self.data_dir):
            if not isfile(join(self.data_dir, filename)):
                data_labels.append(filename)
        return data_labels

    def get_data_paths(self):
        data_paths = []
        for label in self.data_labels:
            img_lists=[]
            path = join(self.data_dir, label)
            for filename in listdir(path):
                tokens = filename.split('.')
                if tokens[-1] == 'jpg':
                    image_path=join(path, filename)
                    img_lists.append(image_path)
            shuffle(img_lists)
            data_paths.append(img_lists)
        return data_paths
    def get_mini_batches(self, batch_size=6400, image_size=(100, 100), allchannel=True):
        images = []
        labels = []
        empty=False
        counter=0
        each_batch_size=int(batch_size/len(self.data_info))
        while True:
            for i in range(len(self.data_labels)):
                label = np.zeros(len(self.data_labels),dtype=int)
                label[i] = 1
                if len(self.data_info[i]) < counter+1:
                    empty=True
                    continue
                empty=False
                img = cv2.imread(self.data_info[i][counter])
                if not allchannel:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    img = np.reshape(img, (img.shape[0], img.shape[1], 1))
                images.append(img)
                labels.append(label)
            counter+=1
            if empty:
                break
            # if the iterator is multiple of batch size return the mini batch
            if (counter)%each_batch_size == 0:
                yield np.array(images,dtype=np.uint8), np.array(labels,dtype=np.uint8)
                del images
                del labels
                images=[]
                labels=[]
    
    
    
class NetworkBuilder: 
    def __init__(self): 
        pass
    def attach_conv_layer(self, input_layer, output_size, feature_size, strides, padding='SAME',
                          summary=False):
        with tf.name_scope("Convolution") as scope:
            input_size = input_layer.get_shape().as_list()[-1]
            weights = tf.Variable(tf.random_normal([feature_size[0], feature_size[1], input_size, output_size]), name='conv_weights')
            if summary:
                tf.summary.histogram(weights.name, weights)
            biases = tf.Variable(tf.random_normal([output_size]),name='conv_biases')
            conv = tf.nn.conv2d(input_layer, weights, strides=strides, padding=padding)+biases
            return conv
    
    def attach_pooling_layer(self, input_layer, ksize, strides, padding='SAME'):
        with tf.name_scope("Pooling") as scope:
            return tf.nn.max_pool(input_layer, ksize=ksize, strides=strides, padding=padding)
        
        
    def flatten(self, input_layer):
        with tf.name_scope("Flatten") as scope:
            input_size = input_layer.get_shape().as_list()
            new_size = input_size[-1]*input_size[-2]*input_size[-3]
            return tf.reshape(input_layer, [-1, new_size])

    
    def attach_relu_layer(self, input_layer):
        with tf.name_scope("Activation") as scope:
            return tf.nn.relu(input_layer)

nb=NetworkBuilder()
    
input_img = tf.placeholder(tf.float32, shape=[None, 100, 100, 1], name="input")
target_labels = tf.placeholder(tf.int64, shape=[None,64], name="Targets")

with tf.name_scope("ModelV2") as scope:
    
    model = input_img
    model1 = nb.attach_conv_layer( model, output_size=4, feature_size=(5,5), strides=[1,2,2,1], padding='SAME',
                          summary=False)
    model1 = nb.attach_pooling_layer(input_layer=model1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

    model2 = nb.attach_conv_layer( model1, output_size=16, feature_size=(5,5), strides=[1,2,2,1], padding='SAME',
                          summary=False)
    model2 = nb.attach_pooling_layer(input_layer=model2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

    model3 = nb.attach_conv_layer( model2, output_size=32, feature_size=(5,5), strides=[1,2,2,1], padding='SAME',
                          summary=False)
    model3 = nb.attach_pooling_layer(input_layer=model3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

    model4 = nb.attach_conv_layer( model3, output_size=64, feature_size=(5,5), strides=[1,2,2,1], padding='SAME',
                          summary=False)
    model4 = nb.attach_pooling_layer(input_layer=model4, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    fc1 = nb.flatten(model4)
    fc2 = nb.attach_relu_layer(fc1)
    fc3  = tf.nn.dropout(fc2,keep_prob=0.8)
    prediction = tf.nn.softmax(fc3)
    
epochs = 20
batchSize = 6400


with tf.name_scope("Optimization") as scope:
    global_step = tf.Variable(0, name='global_step', trainable=False)
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=fc3, labels=target_labels)
    cost = tf.reduce_mean(cross_entropy)
    tf.summary.scalar("cost", cost)

with tf.name_scope('accuracy') as scope:
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(target_labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

dg = DataSetGenerator("./fruit/Training")  

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for epoch in range(epochs):
        batches = dg.get_mini_batches(batchSize,(100,100), allchannel=False)
        for imgs ,labels in batches:
            imgs=np.divide(imgs, 255)
            error, acu = sess.run([cost, accuracy],
                                            feed_dict={input_img: imgs, target_labels: labels})
            print("epoch=", epoch, "err=", error, "accuracy=", acu)
    
    
    
    
    
    
    
    
    
    
    
    

epoch= 0 err= 7698.62 accuracy= 0.01140625
epoch= 0 err= 7744.3467 accuracy= 0.01390625
epoch= 0 err= 7721.1577 accuracy= 0.0115625
epoch= 0 err= 7759.4263 accuracy= 0.013125
epoch= 1 err= 7728.9556 accuracy= 0.01203125
epoch= 1 err= 7733.726 accuracy= 0.01484375
epoch= 1 err= 7732.635 accuracy= 0.01125
epoch= 1 err= 7719.696 accuracy= 0.01421875
epoch= 2 err= 7727.689 accuracy= 0.011875
epoch= 2 err= 7720.292 accuracy= 0.01265625
epoch= 2 err= 7739.0312 accuracy= 0.0115625
epoch= 2 err= 7751.301 accuracy= 0.01234375
epoch= 3 err= 7729.0 accuracy= 0.0128125
epoch= 3 err= 7706.5625 accuracy= 0.014375
epoch= 3 err= 7721.0586 accuracy= 0.01109375
epoch= 3 err= 7755.025 accuracy= 0.011875
epoch= 4 err= 7710.2593 accuracy= 0.0115625
epoch= 4 err= 7733.8496 accuracy= 0.01265625
epoch= 4 err= 7680.9644 accuracy= 0.01203125
epoch= 4 err= 7737.3555 accuracy= 0.01390625
epoch= 5 err= 7730.105 accuracy= 0.013125
epoch= 5 err= 7703.3125 accuracy= 0.01515625
epoch= 5 err= 7737.3594 accuracy= 0.0109